<a href="https://colab.research.google.com/github/seo-won-lee/DialectTranslater/blob/main/final_performance_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. dataset 준비

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Dacos/dataset/aug-test.csv', encoding='ISO-8859-1')

In [ ]:
data.head(10)

,source,target,country,source_word
0,Can you believe how good the busy was?,Can you believe how good the My call didnât ...,British,busy
1,They said they needed more wait in line for th...,They said they needed more queue up for the pa...,British,wait in line
2,Is that your beat a dead horse parked in the d...,Is that your flog a dead horse parked in the d...,British,beat a dead horse
3,We decided to buy a hoodie for the road trip.,We decided to buy a bunny hug for the road trip.,Canada,hoodie
4,He sold his old to let and bought a new one.,He sold his old for rent and bought a new one.,British,to let
5,We had a debate about whether to call them app...,We had a debate about whether to call them whi...,British,appliances
6,I love eating driveway during movie nights.,I love eating drive-thru during movie nights.,British,driveway
7,I really enjoyed the doorman this morning.,I really enjoyed the She greeted the porter as...,British,doorman
8,A expression is a great option for camping trips.,A phrase is a great option for camping trips.,British,expression
9,"In Australia, we often call pacifier 'pacifier'.","In Australia, we often call dummy 'dummy'.",British,pacifier


In [ ]:
# 데이터프레임 생성
df = pd.DataFrame(data)

- references: target
- input: source
- ouutputs: 모델이 예측 값 (encoding 상태)
- predictions: 디코딩된 output 형태

In [ ]:
references = df["target"].tolist()
inputs = df["source"].tolist()

In [ ]:
references[:10]

['Can you believe how good the My call didnâ\x80\x99t go through because the line was engaged. was?',
 'They said they needed more queue up for the party.',
 'Is that your flog a dead horse parked in the driveway?',
 'We decided to buy a bunny hug for the road trip.',
 'He sold his old for rent and bought a new one.',
 'We had a debate about whether to call them white goods or white goods.',
 'I love eating drive-thru during movie nights.',
 'I really enjoyed the She greeted the porter as she walked into the building. this morning.',
 'A phrase is a great option for camping trips.',
 "In Australia, we often call dummy 'dummy'."]

- 데이터 토큰화 함수 정의

In [ ]:
import string

In [ ]:
# 번역 생성 함수 정의
def preprocess(sentence):
    ## 사전 처리 요구사항
    # Tokenization
    tokens = sentence.split()
    # Case Normalization
    tokens = [token.lower() for token in tokens]
    # Punctuation Removal
    tokens = [token.translate(str.maketrans('', '', string.punctuation)) for token in tokens]
    return tokens

In [ ]:
ref_tokens = [preprocess(sentence) for sentence in references]
input_tokens = [preprocess(sentence) for sentence in inputs]

### 2. huggingface 에서 model import

In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 모델 이름 설정
model_name1 = "thisischloe/augv1-dialect-Translater"
model_name2 = "thisischloe/augv2-dialect-Translater"
model_name3 = "thisischloe/augv3-dialect-Translater"

# 토크나이저와 모델 로드
tokenizer1 = AutoTokenizer.from_pretrained(model_name1)
model1 = AutoModelForSeq2SeqLM.from_pretrained(model_name1)

tokenizer2 = AutoTokenizer.from_pretrained(model_name2)
model2 = AutoModelForSeq2SeqLM.from_pretrained(model_name2)

tokenizer3 = AutoTokenizer.from_pretrained(model_name3)
model3 = AutoModelForSeq2SeqLM.from_pretrained(model_name3)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

### 3. model prediction(output) 도출

In [ ]:
# 모델 입력 준비
encoded_inputs1 = tokenizer1(inputs, return_tensors="pt", padding=True, truncation=True)
encoded_inputs2 = tokenizer2(inputs, return_tensors="pt", padding=True, truncation=True)
encoded_inputs3 = tokenizer3(inputs, return_tensors="pt", padding=True, truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# 모델 추론
outputs1 = model1.generate(**encoded_inputs1)
outputs2 = model2.generate(**encoded_inputs2)
outputs3 = model3.generate(**encoded_inputs3)

In [ ]:
# 디코딩된 예측 생성
predictions1 = [tokenizer1.decode(output, skip_special_tokens=True) for output in outputs1]
predictions2 = [tokenizer2.decode(output, skip_special_tokens=True) for output in outputs2]
predictions3 = [tokenizer3.decode(output, skip_special_tokens=True) for output in outputs3]

In [ ]:
pred_tokens1 = [preprocess(sentence) for sentence in predictions1]
pred_tokens2 = [preprocess(sentence) for sentence in predictions2]
pred_tokens3 = [preprocess(sentence) for sentence in predictions3]

In [ ]:
# pred_tokens[:10]

[['can', 'you', 'bring', 'me', 'a', 'tough', 'titty', 'from', 'the', 'store'],
 ['we',
  'use',
  'a',
  'service',
  'station',
  'every',
  'day',
  'in',
  'our',
  'house'],
 ['can', 'you', 'bring', 'me', 'a', 'bathroom', 'from', 'the', 'store'],
 ['i', 'love', 'editorial', 'with', 'tomato', 'sauce'],
 ['can', 'you', 'bring', 'me', 'a', 'can', 'from', 'the', 'store'],
 ['we', 'use', 'a', 'movie', 'theater', 'every', 'day', 'in', 'our', 'house'],
 ['the', 'overpass', 'was', 'left', 'on', 'the', 'table'],
 ['we', 'use', 'a', 'soda', 'every', 'day', 'in', 'our', 'house'],
 ['i', 'learned', 'is', 'very', 'popular', 'in', 'this', 'country'],
 ['the', 'flashlight', 'was', 'left', 'on', 'the', 'table']]

### 4.1 성능평가1
### SSA (Semantic Substitution Accuracy)

SSA (Semantic Substitution Accuracy)
- 의미적으로 정확히 대체된 단어 비율 평가
- SSA = 의미적으로 정확히 대체된 단어수/전체 대체된 단어수

In [ ]:
from nltk.corpus import wordnet
from typing import List

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

- 두단어가 의미적으로 동등한지 평가하는 함수 정의
- WordNet Synsets를 사용 -> 단어 간 유사성 비교

In [ ]:
def is_semantically_equivalent(word1: str, word2: str) -> bool:

    synsets1 = wordnet.synsets(word1)
    synsets2 = wordnet.synsets(word2)

    # 두 단어 중 하나라도 Synset이 없으면 False
    if not synsets1 or not synsets2:
        return False

    # Synset 간 최대 유사도 계산
    max_similarity = max(s1.wup_similarity(s2) or 0 for s1 in synsets1 for s2 in synsets2)
    return max_similarity > 0.8  # 임계값(threshold)을 0.8로 설정

- SSA 계산 함수 -> 대체 작업 당 동등한 의미 단위 대체 확인

In [ ]:
def calculate_ssa(reference: str, prediction: str) -> float:

    ref_tokens = reference.split()
    pred_tokens = prediction.split()

    # 두 문장의 토큰 수가 다르면 패딩 처리
    if len(ref_tokens) != len(pred_tokens):
        max_len = max(len(ref_tokens), len(pred_tokens))
        ref_tokens += [''] * (max_len - len(ref_tokens))
        pred_tokens += [''] * (max_len - len(pred_tokens))

    # 의미적으로 동등한 대체 작업 평가
    correct_substitutions = sum(
        1 for ref_word, pred_word in zip(ref_tokens, pred_tokens)
        if is_semantically_equivalent(ref_word, pred_word)
    )

    # 전체 대체 작업 수
    total_substitutions = len(ref_tokens)

    # SSA 계산
    return correct_substitutions / total_substitutions if total_substitutions > 0 else 0.0

- 리스트 내부 모든 문장에 대해 SSA 계산하는 함수

In [ ]:
def evaluate_ssa(reference_sentences, prediction_sentences):
    ssa_scores = []
    for ref, pred in zip(reference_sentences, prediction_sentences):
        ssa = calculate_ssa(ref, pred)
        ssa_scores.append(ssa)
    return ssa_scores

In [ ]:
def calculate_average_ssa(ssa_scores):

    return sum(ssa_scores) / len(ssa_scores) if ssa_scores else 0.0

In [ ]:
# SSA 점수 계산
ssa_scores1 = evaluate_ssa(references, predictions1)
ssa_scores2 = evaluate_ssa(references, predictions2)
ssa_scores3 = evaluate_ssa(references, predictions3)

# SSA 평균 점수 계산
average_ssa1 = calculate_average_ssa(ssa_scores1)
average_ssa2 = calculate_average_ssa(ssa_scores2)
average_ssa3 = calculate_average_ssa(ssa_scores3)

# 결과 출력
print(f"Average SSA 1: {average_ssa1:.2f}")
print(f"Average SSA 2: {average_ssa2:.2f}")
print(f"Average SSA 3: {average_ssa3:.2f}")

Average SSA 1: 0.24
Average SSA 2: 0.25
Average SSA 3: 0.16


- ssa 점수를 실제 문장과 비교

In [ ]:
# for ref, pred, ssa in zip(references[5:10], predictions[5:10], ssa_scores):
#     print(f"ref: {ref}")
#     print(f"pred: {pred}")
#     print(f"SSA: {ssa:.2f}")
#     print()

ref: We use a movies every day in our house.
pred: We use a movie theater every day in our house.
SSA: 0.50

ref: The flyover was left on the table.
pred: The overpass was left on the table.
SSA: 0.20

ref: We use a fizzy drink every day in our house.
pred: We use a soda every day in our house.
SSA: 0.56

ref: learnt is very popular in this country.
pred: I learned is very popular in this country.
SSA: 0.17

ref: The torch was left on the table.
pred: The flashlight was left on the table.
SSA: 0.56



### 4.2 성능평가2
### TER (Translation Edit Rate)

Translation Edit Rate (TER)
- 기계번역 결과 품질 평가 지표
- 삽입/삭제/대체/단어 재정렬 등의 편집 작업의 최소 횟수를 기준으로 번역의 품질 평가
- TER = 편집 작업의 총합 / 참조 번역 단어 수

In [ ]:
!pip install pyter3

In [ ]:
import pyter

In [ ]:
# TER 계산 함수 정의
def calculate_average_ter(references, predictions):
    ter_scores = []
    for ref, pred in zip(references, predictions):
        ter = pyter.ter(ref, pred)
        ter_scores.append(ter)

    # TER 점수들의 평균을 계산
    average_ter = sum(ter_scores) / len(ter_scores) if ter_scores else 0.0
    return average_ter

In [ ]:
print(f"TER score 1: {calculate_average_ter(references, predictions1):.4f}")
print(f"TER score 2: {calculate_average_ter(references, predictions2):.4f}")
print(f"TER score 3: {calculate_average_ter(references, predictions3):.4f}")

TER score 1: 0.4364
TER score 2: 0.4233
TER score 3: 0.4340


### 4.3 성능평가3
### STS (Semantic Textual Similarity)

- BERT 모델을 활용한 BERTScore 측정
- 의미적 유사성 반영 가능
- 어휘적 차이도 반영

In [ ]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
from bert_score import score

In [ ]:
for i, pred in enumerate([predictions1, predictions2, predictions3], start=1):
    # BERTScore 계산
    P, R, F1 = score(pred, references, lang="en")

    # Precision, Recall, F1의 평균 계산
    avg_precision = P.mean()
    avg_recall = R.mean()
    avg_f1 = F1.mean()

    # 결과 출력
    print(f"BERT Average Precision ({i}): {avg_precision:.4f}")
    print(f"BERT Average Recall ({i}): {avg_recall:.4f}")
    print(f"BERT Average F1 ({i}): {avg_f1:.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

BERT Average Precision (1): 0.9480
BERT Average Recall (1): 0.9356
BERT Average F1 (1): 0.9417


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT Average Precision (2): 0.9461
BERT Average Recall (2): 0.9363
BERT Average F1 (2): 0.9410


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT Average Precision (3): 0.9326
BERT Average Recall (3): 0.9272
BERT Average F1 (3): 0.9298


### 4.4 성능평가4
### Word-Level-Accuracy

In [ ]:
import string

In [ ]:
# 번역 생성 함수 정의
def preprocess(sentence):
    ## 사전 처리 요구사항
    # Tokenization
    tokens = sentence.split()
    # Case Normalization
    tokens = [token.lower() for token in tokens]
    # Punctuation Removal
    tokens = [token.translate(str.maketrans('', '', string.punctuation)) for token in tokens]
    return tokens

In [ ]:
def word_level_accuracy(test_data, model, tokenizer):
    total_words = 0
    correct_words = 0

    for _, row in test_data.iterrows():
        source = row['source']
        target = row['target']
        country = row['country']  # Country 정보 추가

        # 프롬프트 구성
        source_with_country = f"translate American to {country}: {source}"

        # 전처리
        source_tokens = preprocess(source_with_country)
        target_tokens = preprocess(target)

        # 예측 생성
        inputs = tokenizer(source_with_country, return_tensors="pt", truncation=True, max_length=512).to(model.device)
        outputs = model.generate(**inputs)
        predicted = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predicted_tokens = preprocess(predicted)

        # Word-level accuracy 계산
        total_words += len(target_tokens)
        correct_words += sum([1 for t, p in zip(target_tokens, predicted_tokens[:len(target_tokens)]) if t == p])

    # Accuracy 계산
    accuracy = correct_words / total_words if total_words > 0 else 0
    return accuracy


In [ ]:
# word-level accuracy 결과
accuracy1 = word_level_accuracy(data, model1, tokenizer1)
accuracy2 = word_level_accuracy(data, model2, tokenizer2)
accuracy3 = word_level_accuracy(data, model3, tokenizer3)

print(f"Word-Level Accuracy1: {accuracy1:.2%}")
print(f"Word-Level Accuracy2: {accuracy2:.2%}")
print(f"Word-Level Accuracy3: {accuracy3:.2%}")

Word-Level Accuracy1: 61.75%
Word-Level Accuracy2: 62.14%
Word-Level Accuracy3: 63.01%
